<a href="https://colab.research.google.com/github/Evelynmaezj/ANALITICA-DE-DATOS/blob/main/Actividad%203.1/act3_9_Geneva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalamos la libreria de STREAMLIT
%pip install streamlit

In [ ]:
#Instalamos la libreria de PLOTLY
%pip install plotly

In [ ]:
#Instalamos librería para análisis de variables categóricas
%pip install funpymodeling

In [ ]:
#Instalamos el tunnel local (node.js)
#Para instalar npm en visual studio
#1.Desde Google escribir node.js
#2. Instalar la versión más recomendada
!npm install localtunnel


up to date, audited 23 packages in 927ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
#Cargar archivo csv desde equipo
from google.colab import files
files.upload()

In [ ]:
%%writefile app.py
#Creamos el archivo de la APP en el interprete principal (Python)

##########IMPLEMENTACION DE DASHBOARD##########

#Verificamos que todas las librerias se puedan importar
#Importamos librerias
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score

#Definimos la instancia de streamlit
@st.cache_resource

#Creamos la funcion de carga de datos
def load_data():
    #Carga desde un archivo csv con indice
    data = pd.read_csv('geneva_listing_clean.csv',encoding='utf-8',index_col=0)

    #Etapa de procesamiento de Datos#

    #Obtener Frames para regresion no lineal
    #Se crea una copia del frame original
    data_cuant = data.copy()

    #Se eliminan columnas innecesarias
    data_cuant.drop(['last_scraped','source','name','description','host_name','host_since','host_location','host_response_time','host_verifications','host_has_profile_pic','host_identity_verified','neighbourhood_cleansed','property_type','bathrooms_text','amenities','has_availability','calendar_last_scraped','first_review','last_review','host_is_superhost','room_type','instant_bookable'],axis=1,inplace=True)

    #Encontramos todos los coeficientes de correlacion entre las variables
    corr_f = data_cuant.corr()

    #Encontramos el valor absoluto de todas las correlaciones entre las variables
    corr_factors = abs(corr_f)

    #Variables a analizar
    data_results = data_cuant[['host_response_rate', 'host_acceptance_rate', 'host_total_listings_count', 'accommodates', 'reviews_per_month', 'price']]

    #Selecciono las columnas tipo numericas del dataframe
    numeric_data = data_cuant.select_dtypes(['float','int'])  #Devuelve columnas
    numeric_cols = numeric_data.columns                         #Devuelve lista de columnas

    return data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols

#Cargo los datos obtenidos de la funcion 'load_data'
data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols = load_data()

#Funiones para modelos No Lineales
#y = ax^2 + bx + c   ("Función cuadrática")
def funcCuad(x,a,b,c):
  return a*x**2 + b*x + c

#y = a*np.abs(x) + b*x + c (Función Valor absoluto)
def funcAbs(x,a,b,c):
  return a*np.abs(x) + b*x + c

#y = a*exp(-bx) + c   ("Función exponencial")
def funcExp(x,a,b,c):
  return a*np.exp(-b*x) + c

#y = a*np.log(x) + b  (Función logaritmica)
def funcLog(x,a,b,c):
  return a*np.log(x) + b

#PAGINA PRINCIPAL - Titulo
st.title('AIRBNB - ANALISIS DE DATOS')
st.header('Regresion No Lineal')

#SIDEBAR - Seleccion de Pagina
st.sidebar.title('MENU')
st.sidebar.header('Seleccione una Ciudad')
city = st.sidebar.selectbox(label='Ciudad',options=['','Mexico','Geneva','Boston'])

#CONTENIDO - Ciudad Mexico
if city == 'Mexico':
    st.subheader('Mexico')

#CONTENIDO - Ciudad Boston
if city == 'Boston':
    st.subheader('Boston')

#CONTENIDO - Ciudad Geneva
if city == 'Geneva':
    st.subheader('Geneva')

    #Generamos un cuadro de seleccion (Checkbox) en una barra lateral (sidebar) para mostrar dataset
    check_box = st.sidebar.checkbox(label="Mostrar Correlaciones Lineales")

    if check_box:
        #Mostramos el dataset
        st.write(corr_factors)

    #Generamos un cuadro de seleccion (Checkbox) en una barra lateral (sidebar) para mostrar dataset
    check_map = st.sidebar.checkbox(label="Mostrar Heat Map")

    if check_map:
        #Mostramos el dataset
        heatmap = px.imshow(corr_factors)

        # Display the heatmap in Streamlit
        st.plotly_chart(heatmap)

    st.text('Seleccione una Variable Objetivo para graficar')
    vars_result_selected = st.selectbox(label='Variables Objetivo',options=data_results.columns)

    st.sidebar.subheader('Criterios de Graficación')
    x_var = st.sidebar.selectbox(label='Seleccione Variable X',options=numeric_cols)
    func = st.sidebar.selectbox(label='Seleccione Funcion para graficar',options=['','Cuadratica','Absoluto','Exponencial','Logaritmica'])

    if func == 'Cuadratica':
        data_copy = data_cuant.copy()

        #Declaramos las variables dependientes e independientes para la regresion No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Funcion Cuadratica - y = ax^2 + bx + c')

        try:
            #Ajustamos los parametros de la funcion curve_fit
            parametros, covs = curve_fit(funcCuad, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            #Predicciones
            y_pred = funcCuad(x, *parametros)
            data_copy['y_pred'] = y_pred

            #Calclamos el coeficiete de determinación del modelo
            r2 = r2_score(y, y_pred)

            #Calculamos el coeficiente de correlacion del modelo
            r = np.sqrt(abs(r2))

            #Mostramos el Coeficiente
            st.text('Coeficiente de Correlacion:')
            st.text(str(r))

            #SCATTERPLOT
            #Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected,'y_pred'],
                                title="Regresion No Lineal - Cuadratica")
            st.plotly_chart(figureRNL)
        except:
            st.text('Es imposible formular la Grafica')

    if func == 'Absoluto':
        data_copy = data_cuant.copy()

        #Declaramos las variables dependientes e independientes para la regresion No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Funcion Valor Absoluto - y = a*np.abs(x) + b*x + c')

        try:
            #Ajustamos los parametros de la funcion curve_fit
            parametros, covs = curve_fit(funcAbs, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            #Predicciones
            y_pred = funcAbs(x, *parametros)
            data_copy['y_pred'] = y_pred

            #Calculamos el coeficiete de determinación del modelo
            r2 = r2_score(y, y_pred)

            #Calculamos el coeficiente de correlacion del modelo
            r = np.sqrt(abs(r2))

            #Mostramos el Coeficiente
            st.text('Coeficiente de Correlacion:')
            st.text(str(r))

            #SCATTERPLOT
            #Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected,'y_pred'],
                                title="Regresion No Lineal - Valor Absoluto")
            st.plotly_chart(figureRNL)
        except:
            st.text('Es imposible formular la Grafica')

    if func == 'Exponencial':
        data_copy = data_cuant.copy()

        #Declaramos las variables dependientes e independientes para la regresion No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Funcion Exponencial - y = a*exp(-bx) + c')

        try:
            #Ajustamos los parametros de la funcion curve_fit
            parametros, covs = curve_fit(funcExp, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            #Predicciones
            y_pred = funcExp(x, *parametros)
            data_copy['y_pred'] = y_pred

            #Calclamos el coeficiete de determinación del modelo
            r2 = r2_score(y, y_pred)

            #Calculamos el coeficiente de correlacion del modelo
            r = np.sqrt(abs(r2))

            #Mostramos el Coeficiente
            st.text('Coeficiente de Correlacion:')
            st.text(str(r))

            #SCATTERPLOT
            #Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected,'y_pred'],
                                title="Regresion No Lineal - Exponencial")
            st.plotly_chart(figureRNL, color='pink',)
        except:
            st.text('Es imposible formular la Grafica')

    if func == 'Logaritmica':
        data_copy = data_cuant.copy()

        #Declaramos las variables dependientes e independientes para la regresion No Lineal
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]

        st.text('Funcion Logaritmica - y = a*np.log(x) + b')

        try:
            #Ajustamos los parametros de la funcion curve_fit
            parametros, covs = curve_fit(funcLog, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)

            #Predicciones
            y_pred = funcLog(x, *parametros)
            data_copy['y_pred'] = y_pred

            #Calclamos el coeficiete de determinación del modelo
            r2 = r2_score(y, y_pred)

            #Calculamos el coeficiente de correlacion del modelo
            r = np.sqrt(abs(r2))

            #Mostramos el Coeficiente
            st.text('Coeficiente de Correlacion:')
            st.text(str(r))

            #SCATTERPLOT
            #Despliegue de un scatter plot, definiendo las variables "X" y "Y"
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected,'y_pred'],
                                title="Regresion No Lineal - Logaritmica")
            st.plotly_chart(figureRNL)
        except:
            st.text('Es imposible formular la Grafica')



Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.237.118.34



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.118.34:8501

your url is: https://loud-bikes-heal.loca.lt
/content/app.py:197: OptimizeWarning:

Covariance of the parameters could not be estimated

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

overflow encountered in exp

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

overflow encountered in exp

2024-10-17 00:24:37.142 Session with id 0718cd03-cea3-468d-ab29-f92af1bc71b3 is already connected! Connecting to a new session.
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning:

overflow encountered in exp

/content/app.py:232: OptimizeWarning:

Covariance of the parameters could not be estimated

/content/app.py:232: OptimizeWarning:

Covariance of the parameters could not be estimated

/content/app.